# Sentence-level analysis on a single book: 

In [1]:
import nltk
import re
import pandas as pd

import numpy as np
from scipy.stats import mode
# for cosine distance calculation:
from scipy import spatial

import sklearn.feature_extraction.text as text
from sklearn import decomposition

import matplotlib.pyplot as plt
import seaborn as sns

from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from nltk.tokenize import wordpunct_tokenize

import pickle

In [4]:
#unpickle

pickloader=open( "lemmed_books.pkl", "rb" )
clean_books = pickle.load(pickloader)
pickloader.close()

pickloader=open("book_titles.pkl","rb")
titles = pickle.load(pickloader)
pickloader.close()

In [7]:
clean_books[5][0:1000]

"the time machine , by h . g . well [ 1898 ] i the time traveller ( for so it will be convenient to speak of him ) wa expounding a recondite matter to u . his grey eye shone and twinkled , and his usually pale face wa flushed and animated . the fire burned brightly , and the soft radiance of the incandescent light in the lily of silver caught the bubble that flashed and passed in our glass . our chair , being his patent , embraced and caressed u rather than submitted to be sat upon , and there wa that luxurious after - dinner atmosphere when thought roams gracefully free of the trammel of precision . and he put it to u in this way -- marking the point with a lean forefinger -- a we sat and lazily admired his earnestness over this new paradox ( a we thought it ) and his fecundity . ' you must follow me carefully . i shall have to controvert one or two idea that are almost universally accepted . the geometry , for instance , they taught you at school is founded on a misconception .' ' is

In [8]:
pickler = open("novel_names_dash.pkl","wb")
pickle.dump(list(titles), pickler)
pickler.close()

In [9]:
clean_books = [book.lower() for book in clean_books]

In [12]:
#I did this already; this is redundant code in case you're starting from preprocessing

# Lemming: gentler than stemming 
lemma=nltk.stem.WordNetLemmatizer()

lemma_books = []
for book in clean_books:
    lemma_book = [lemma.lemmatize(word) for word in wordpunct_tokenize(book)]
    lemma_book = (' ').join(lemma_book)
    lemma_books.append(lemma_book)

lemma_books[0][0:100]

clean_books = lemma_books

#contractions
import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "i would",
  "i'd've": "i would have",
  "i'll": "i will",
  "i'll've": "i will have",
  "i'm": "i am",
  "i've": "i have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

expandContractions(clean_books[0].lower())

clean_books_expanded = [expandContractions(book.lower()) for book in clean_books]

clean_books = clean_books_expanded

In [10]:
#tokenize as sentences for sentiment analysis

clean_book_sentences = []
for book in clean_books:
    sentences = sent_tokenize(book)
    clean_book_sentences.append(sentences)

In [ ]:
pickler = open("clean_book_sentences.pkl","wb")
pickle.dump(clean_book_sentences, pickler)
pickler.close()

### Function for creating an average sentiment polarity for each bin:

In [ ]:
clean_book_sentences[0][2]

In [ ]:
sentiments = [TextBlob(sentence).sentiment.polarity for sentence in clean_book_sentences[novel_index]]
sentiments[:5] #etc

In [ ]:
#you might want to play with this depending on article/book/text format
number_of_buckets = 5 

#Novel has Exposition, Rising Action, Climax, Falling Action, Resolution, hence 5

def get_sentiment(novel_index):

    #make list of sentiment by sentences
    sentiments = [TextBlob(sentence).sentiment.polarity for sentence in clean_book_sentences[novel_index]]
    
    #bin list into buckets
    sentiments_binned = list(np.array_split(sentiments, number_of_buckets))

    #take the mean of each bucket for graphing
    sentiments_binned = [np.mean(bin) for bin in sentiments_binned]
    
    return(sentiments_binned)

In [ ]:
#now chose a book:
novel_index=0
print('Novel name:', novel_names[novel_index])

In [ ]:
sentiments_binned = get_sentiment(novel_index)

y_pos = np.arange(len(sentiments_binned))
plt.figure(figsize=(10,5))
plt.bar(y_pos, height=sentiments_binned);
plt.ylabel("Sentiment polarity")
plt.xlabel("Book narrative -->")
plt.title(f'Sentiment trajectory for {novel_names[novel_index]}');

### Source for some of this code:
- https://de.dariah.eu/
- https://de.dariah.eu/tatom/topic_model_python.html
- https://de.dariah.eu/tatom/topic_model_visualization.html#topic-model-visualization
- https://github.com/Jack-Etheredge/Project-guten-bag-of-words